# _Development: Nov. 13, 2019_

**_Today's Focus_**

1. Login in to `EarnyBot` account
2. Monitor `EarnyBot` with my original account (i.e. keys/tokens)
    - Set-up listener that monitors `EarnyBot` and downloads specified data to PostgreSQL database
3. Make a new tweet with `EarnyBot`
4. Run listener script to see if new tweet is properly appended.
5. Repeat process but with two new tweets

In [1]:
import dataset
import logging
from joetools import private
import os
import tweepy

In [2]:
logger = logging.getLogger()

In [3]:
# set environment variables
private.environment_variables()

In [4]:
def database_connect():
    '''connects to PostgreSQL database and table (via user input)'''
    print('Please insert table name you would like to reference.')
    table_name = str(input())
    
    try:
        # connect to PostgreSQL database
        db = dataset.connect(os.environ.get('CONNECTION_STRING'))
        # create reference to specified table
        table = db[table_name]
        return table
    except Exception as e:
        print(e)

In [5]:
table = database_connect()

Please insert table name you would like to reference.


 earnybot_nov13


In [6]:
def api_setup():
    '''set up Twitter API'''

    # access tweepy authenticator
    auth = tweepy.OAuthHandler(os.environ.get('TWITTER_API_KEY'), os.environ.get('TWITTER_API_SECRET'))
    auth.set_access_token(os.environ.get('TWITTER_TOKEN'), os.environ.get('TWITTER_TOKEN_SECRET'))

    # pass in above authentication to API
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error('Error creating API', exc_info=True)
        raise e
    logger.info('API created')
    return api

In [7]:
api = api_setup()

In [8]:
username = str(input())

 EarnyBot


In [10]:
for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(5):
    urls = []
    if tweet.entities['urls']:
        url_dict = tweet.entities['urls']
        for n in url_dict:
            urls.append(n['url'])
    else:
        print('No URLs.')

No URLs.
No URLs.
No URLs.
No URLs.


In [11]:
urls

['https://t.co/yy9thIWR4P']

In [12]:
for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(5):
    if tweet.entities['urls']:
        urls = [n['url'] for n in tweet.entities['urls']]
    else:
        print('No URLs.')

No URLs.
No URLs.
No URLs.
No URLs.


In [15]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve attributes from tweet
        created_at = tweet.created_at
        tweet_id = str(tweet.id_str)
        source = tweet.source
        retweet_count = tweet.retweet_count
        favorite_count = tweet.favorite_count
        
        if tweet.entities['hashtags']:
            hashtags = [n['text'] for n in tweet.entities['hashtags']]
        else:
            hashtags = 'None'
        
        if tweet.entities['urls']:
            urls = [n['url'] for n in tweet.entities['urls']]
        else:
            urls = 'None'
            
        text = tweet.full_text
        
        # insert database table reference
        table = table
        # try to insert
        try:
            if tweet_id not in table:
                table.insert(dict(
                    created_at=created_at,
                    tweet_id=tweet_id,
                    source=source,
                    retweet_count=retweet_count,
                    favorite_count=favorite_count,
                    hashtags=hashtags,
                    urls=urls,
                    text=text
                ))
            else:
                pass
        except Exception as e:
            print(e)

In [16]:
gather_tweets(api, username, 100, table)

In [18]:
import pandas as pd

pd.DataFrame(table.all())

,id,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,1,2019-11-11 21:43:59,1194007890757083141,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
1,2,2019-11-11 21:40:54,1194007113628110848,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
2,3,2019-11-11 21:40:45,1194007078349750272,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
3,4,2019-11-11 21:40:37,1194007043230842880,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
4,5,2019-11-11 04:23:05,1193745939795763201,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
5,6,2019-11-10 00:06:48,1193319058227236865,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
6,7,2019-11-10 00:06:46,1193319049054343169,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
7,8,2019-11-10 00:03:51,1193318312471605249,EarnyBot,856,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."
8,9,2019-11-09 23:07:14,1193304066291580931,EarnyBot,0,0,None,None,"""Our life is what our thoughts make it."" -- Ma..."
9,10,2019-11-09 23:07:08,1193304041100644353,EarnyBot,0,0,None,None,"""You can commit injustice by doing nothing."" -..."


In [19]:
gather_tweets(api, username, 100, table)

In [20]:
pd.DataFrame(table.all())

,id,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,1,2019-11-11 21:43:59,1194007890757083141,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
1,2,2019-11-11 21:40:54,1194007113628110848,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
2,3,2019-11-11 21:40:45,1194007078349750272,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
3,4,2019-11-11 21:40:37,1194007043230842880,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
4,5,2019-11-11 04:23:05,1193745939795763201,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
5,6,2019-11-10 00:06:48,1193319058227236865,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
6,7,2019-11-10 00:06:46,1193319049054343169,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
7,8,2019-11-10 00:03:51,1193318312471605249,EarnyBot,856,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."
8,9,2019-11-09 23:07:14,1193304066291580931,EarnyBot,0,0,None,None,"""Our life is what our thoughts make it."" -- Ma..."
9,10,2019-11-09 23:07:08,1193304041100644353,EarnyBot,0,0,None,None,"""You can commit injustice by doing nothing."" -..."


In [21]:
gather_tweets(api, username, 100, table)

In [22]:
pd.DataFrame(table.all())

,id,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,1,2019-11-11 21:43:59,1194007890757083141,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
1,2,2019-11-11 21:40:54,1194007113628110848,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
2,3,2019-11-11 21:40:45,1194007078349750272,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
3,4,2019-11-11 21:40:37,1194007043230842880,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
4,5,2019-11-11 04:23:05,1193745939795763201,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
5,6,2019-11-10 00:06:48,1193319058227236865,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
6,7,2019-11-10 00:06:46,1193319049054343169,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
7,8,2019-11-10 00:03:51,1193318312471605249,EarnyBot,856,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."
8,9,2019-11-09 23:07:14,1193304066291580931,EarnyBot,0,0,None,None,"""Our life is what our thoughts make it."" -- Ma..."
9,10,2019-11-09 23:07:08,1193304041100644353,EarnyBot,0,0,None,None,"""You can commit injustice by doing nothing."" -..."


In [24]:
table.create_index(columns='tweet_id', name='tweet_id')

In [25]:
pd.DataFrame(table.all())

,id,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,1,2019-11-11 21:43:59,1194007890757083141,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
1,2,2019-11-11 21:40:54,1194007113628110848,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
2,3,2019-11-11 21:40:45,1194007078349750272,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
3,4,2019-11-11 21:40:37,1194007043230842880,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
4,5,2019-11-11 04:23:05,1193745939795763201,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
5,6,2019-11-10 00:06:48,1193319058227236865,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
6,7,2019-11-10 00:06:46,1193319049054343169,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
7,8,2019-11-10 00:03:51,1193318312471605249,EarnyBot,856,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."
8,9,2019-11-09 23:07:14,1193304066291580931,EarnyBot,0,0,None,None,"""Our life is what our thoughts make it."" -- Ma..."
9,10,2019-11-09 23:07:08,1193304041100644353,EarnyBot,0,0,None,None,"""You can commit injustice by doing nothing."" -..."


In [27]:
table.drop()

In [29]:
pd.DataFrame(table.all())

""


## _Second Attempt_

In [30]:
def database_connect():
    '''connects to PostgreSQL database and table (via user input)'''
    print('Please insert table name you would like to reference.')
    table_name = str(input())
    
    try:
        # connect to PostgreSQL database
        db = dataset.connect(os.environ.get('CONNECTION_STRING'))
        # create reference to specified table
        table = db.create_table(table_name, primary_id='tweet_id', primary_type=db.types.text)
        return table
    except Exception as e:
        print(e)

In [43]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve attributes from tweet
        tweet_id = str(tweet.id_str)
        created_at = tweet.created_at
        source = tweet.source
        retweet_count = tweet.retweet_count
        favorite_count = tweet.favorite_count
        
        if tweet.entities['hashtags']:
            hashtags = [n['text'] for n in tweet.entities['hashtags']]
        else:
            hashtags = 'None'
        
        if tweet.entities['urls']:
            urls = [n['url'] for n in tweet.entities['urls']]
        else:
            urls = 'None'
            
        text = tweet.full_text
        
        # insert database table reference
        table = table
        # try to insert
        try:
            if tweet_id not in table:
                table.insert(dict(
                    tweet_id=tweet_id,
                    created_at=created_at,
                    source=source,
                    retweet_count=retweet_count,
                    favorite_count=favorite_count,
                    hashtags=hashtags,
                    urls=urls,
                    text=text
                ))
            else:
                pass
        except Exception as e:
            print(e)

In [38]:
second_table = database_connect()

Please insert table name you would like to reference.


 earnybot_rd2


In [39]:
gather_tweets(api, username, 100, second_table)

In [40]:
pd.DataFrame(second_table.all())

,tweet_id,created_at,source,retweet_count,favorite_count,hashtags,urls,text
0,1194765004647673857,2019-11-13 23:52:29,Twitter Web App,0,0,None,None,"""Understand: the future belongs to groups that..."
1,1194763514105286658,2019-11-13 23:46:34,Twitter Web App,0,0,None,None,"""The best fighter is never angry."" -- Lao Tzu"
2,1194007890757083141,2019-11-11 21:43:59,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
3,1194007113628110848,2019-11-11 21:40:54,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
4,1194007078349750272,2019-11-11 21:40:45,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
5,1194007043230842880,2019-11-11 21:40:37,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
6,1193745939795763201,2019-11-11 04:23:05,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
7,1193319058227236865,2019-11-10 00:06:48,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
8,1193319049054343169,2019-11-10 00:06:46,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
9,1193318312471605249,2019-11-10 00:03:51,EarnyBot,856,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."


In [47]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve attributes from tweet
        tweet_id = str(tweet.id_str)
        created_at = tweet.created_at
        source = tweet.source
        retweet_count = tweet.retweet_count
        favorite_count = tweet.favorite_count
        
        if tweet.entities['hashtags']:
            hashtags = [n['text'] for n in tweet.entities['hashtags']]
        else:
            hashtags = 'None'
        
        if tweet.entities['urls']:
            urls = [n['url'] for n in tweet.entities['urls']]
        else:
            urls = 'None'
            
        text = tweet.full_text
        
        # insert database table reference
        table = table
        # try to insert
        try:
            table.upsert(dict(
                tweet_id=tweet_id,
                created_at=created_at,
                source=source,
                retweet_count=retweet_count,
                favorite_count=favorite_count,
                hashtags=hashtags,
                urls=urls,
                text=text), keys=['tweet_id'])
        except Exception as e:
            print(e)

In [48]:
gather_tweets(api, username, 100, second_table)

In [49]:
pd.DataFrame(second_table.all())

,tweet_id,created_at,source,retweet_count,favorite_count,hashtags,urls,text
0,1194765004647673857,2019-11-13 23:52:29,Twitter Web App,0,0,None,None,"""Understand: the future belongs to groups that..."
1,1194763514105286658,2019-11-13 23:46:34,Twitter Web App,0,0,None,None,"""The best fighter is never angry."" -- Lao Tzu"
2,1194007890757083141,2019-11-11 21:43:59,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
3,1194007113628110848,2019-11-11 21:40:54,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
4,1194007078349750272,2019-11-11 21:40:45,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
5,1194007043230842880,2019-11-11 21:40:37,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
6,1193745939795763201,2019-11-11 04:23:05,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
7,1193319058227236865,2019-11-10 00:06:48,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
8,1193319049054343169,2019-11-10 00:06:46,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
9,1193318312471605249,2019-11-10 00:03:51,EarnyBot,856,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."
